In [1]:
import cv2 as cv
import numpy as np
import os
import tensorflow as tf
#import keras
import inkml2img
import re
import json
import xml.etree.ElementTree as ET

In [2]:
# En esta celda se convierten los archivos inkml a .jpg con ayuda del modulo inkml2img
# PATH indica la ubicacion de los archivos .inkml
PATH = r"G:\Documents\CROHME\CROHME\CROHME2013_data\TrainINKML\TrainINKML"
# DIR_PATH indica el directorio donde se guardarán las imagenes
DIR_PATH = r"G:\Documents\CROHME\CROHME\CROHME2013_data\TrainINKML\images"
# se obtiene el nombre de las carpetas que contienen los archivos .inkml
dataSets = os.listdir(PATH)
# se genera una lista con listas vacias (la cantidad de carpetas en dataSets)
arr = [[] for _ in range(len(dataSets))]
print(dataSets)
# se itera por cada carpeta de archivos .inkml
for i in range(len(dataSets)):
    # se agrega a la lista los archivos en determinada carpeta (la de la iteracion actual)
    arr[i] = os.listdir(PATH+'\\'+dataSets[i])
    # se itera por los archivos .inkml de la carpeta
'''for t in arr[i]:
    # si la extension no es .lg
    if ('.lg' not in t):
        # se llama a la funcion inkml2img del modulo hominimo, especificando la ruta del inkml y la ruta a guardar la imagen
        inkml2img.inkml2img(PATH+'\\'+dataSets[i]+'\\'+t,DIR_PATH+'\\'+t[:-5]+'jpg')
        print(t)'''

['expressmatch', 'extension', 'HAMEX', 'KAIST', 'MathBrush', 'MfrDB']


"for t in arr[i]:\n    if ('.lg' not in t):\n        inkml2img.inkml2img(PATH+'\\'+dataSets[i]+'\\'+t,DIR_PATH+'\\'+t[:-5]+'jpg')\n        print(t)"

In [41]:
# en esta celda se obtienen los labels (el ground truth en latex) asociado a cada archivo .inkml
DIR_PATH = r"G:\Documents\CROHME\CROHME\CROHME2013_data\TrainINKML\TrainINKML"
LABELS_PATH = r"G:\Documents\CROHME\CROHME\CROHME2013_data\TrainINKML"
# diccionario donde el ground truth se guardara para cada archivo
items = {}
index = 0
count = 0
# se itera por la cantidad de carpetas que contienen archivos .inkml
for t in arr:
    # se itera por cada archivo inkml
    for i in t:
        if (".lg" not in i):
            # se obtiene el path del archivo concatenando DIR_PATH con el nombre de la carpeta y del archivo
            imgPath = DIR_PATH+"\\"+ dataSets[index]+ "\\"+i
            # dado que .inkml es un tipo de XML, tiene una estructura de arbol, por lo que se utiliza 
            # la funcion ET.parse para hacer uso de esta estructura
            tree = ET.parse(imgPath)
            # se otbiene la raiz del archivo (primera etiqueta en el .inkml)
            root = tree.getroot()
            count =0
            # se itera por cada hijo de la raiz
            for item in root:
                # si el hijo posee texto entonces es posible que contenga el ground truth (label) deseado
                if item.text:
                    # existen tres posibilidades debido a la forma en que fueron codificados los archivos en distintas carpetas
                    # el primero es que el groud truth en latex se encuetre entre simbolos de dolar $
                    if "$" in item.text:
                        current = item.text
                        # se añade como key del diccionario el nombre del archivo y se le asocia el ground truth en latex
                        items[DIR_PATH+"\\"+ dataSets[index]+ "\\"+i] = current[1:-1]
                        break
                    if "\\" in item.text and dataSets[index] == "MathBrush":
                        current = item.text
                        items[DIR_PATH+"\\"+ dataSets[index]+ "\\"+i] = current[1:-1]
                        break
                    # para la carpeta KAIST el latex se encuentra sin elementos externos, por lo que se agrega sin mas
                    if (count ==1 and dataSets[index]=="KAIST" and len(item.text)>2):
                        current = item.text
                        items[DIR_PATH+"\\"+ dataSets[index]+ "\\"+i] = current
                        break
                count +=1
        else:
            print(i)
    index +=1 
# se verifica que el comando log no se encuentre como una secuencia de letras separadas
for key in items:
    if ('l o g' in items[key]):
        # si se encuentra se reemplaza por su respectivo comando en latex correcto
        items[key] = items[key].replace('l o g', '\\log')
# se guarda el diccionario en formato json
with open(r"G:\Documents\CROHME\CROHME\CROHME2013_data\TrainINKML\labels.json", 'w') as f:
    json.dump(items, f, indent=4)

In [59]:
# se realiza un filtrado de los groud truth para separar cada comando, letra y numero presente en el dataset
# no correr, ya que se requiere realizar algunos cambios a mano en el json cuando se ejecuta esta celda

# establece los comandos a borrar, ya que no proporcionan informacion relevante para la ecuacion
delete = ['\\Bigg','\\left','\\right','\\Big','\\mathrm']
# comandos a reemplazar por el comando latex correcto
replace = {'\\to':'\\rightarrow', '\\gt':'>', '\\lt':'<'}
# simbolos a los que se les añade espacios en blanco antes y despues para un mejor tratamiento
add = ['_','{','}','=','(',')','-','+','^','[',']', ',']
count = 0
# se iteran los archivos .inkml, items representa el mismo diccionario guardado en labels.json
for key in items:
    # si existe alguno de los comandos descritos en las listas y diccionario anteriores se realizan la soperaciones
    # correspondientes
    for dele in delete:
        if dele in items[key]:
            items[key] = items[key].replace(dele, "")
    for rep in replace:
        if (rep in items[key]):
            items[key] = items[key].replace(rep, replace[rep])
    for it in add:
        if (it in items[key]):
            items[key] = items[key].replace(it, " "+it+" ")
    # se agrega un espacio en blanco antes de \\ para que pueda diferenciarse a los comandos del resto de letras o numeros
    if ('\\' in items[key]):
        items[key] = items[key].replace('\\', " \\")
    # se separa el ground truth, siendo la condicion para separa el que existan uno o mas espacios en blanco entre caracteres
    items[key] = re.split(r'\s+', items[key])
    count = 0
    # se itera sobre la lista generada de letras, numeros, simbolos y comandos para cada archivo inkml
    # separando los caracteres que se encuentren juntos y no sean parte de un comando, 
    # por ejemplo 'abc' se separa en 'a', 'b', 'c'
    for a in items[key]:
        if ('\\' not in a and len(a)>1):
            uno = items[key][:count]
            dos = re.split('', a)[1:-1]
            tres = items[key][count+1:]
            count+= len(dos)-1
            uno.extend(dos)
            uno.extend(tres)
            items[key] = uno
        count+=1
    if (items[key][-1]==""):
        items[key] = items[key][:-1]
    if (items[key][0]==""):
        items[key] = items[key][1:]
# se guarda en labels.json
with open(r"G:\Documents\CROHME\CROHME\CROHME2013_data\TrainINKML\labels.json", 'w') as f:
    json.dump(items, f, indent=4)

In [2]:
# se instancia la llamada a la funcion Tokenizer de tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# se define el numero maximo de palabras a tokenizar
num_words = 1000
# token para labels desconocidas
oov_token = '<UNK>'
pad_type = 'post'
trunc_type = 'post'

items = {}
# se obtienen las listas de labels (numeros, comandos latex, simbolos y letras) para cada archivo inkml
with open(r"G:\Documents\CROHME\CROHME\CROHME2013_data\TrainINKML\labels.json", 'r') as f:
    items = json.load(f)
tokens = {}
count = 0
# se itera para cada archivo
for key in items:
    # se convierte a string
    items[key] = ' '.join(map(str, items[key]))
    # se anade al inicio y final los labels start y end para indicar inicio y final del ground truth
    items[key] = '<start> '+items[key] + ' <end>'
    # se vuelven a separar
    items[key] = re.sub(r"\s+", " ", items[key])
# se obtiene la cantidad de archvios inkml a tratar
keys = list(items.keys())
# se anade el groud truth de cada archivo a una lista
data = [ items[key] for key in keys ]
print(data[0])
# se instancia la funcion tokenizer con los parametros establecidos en un principio
tokenizer = Tokenizer(num_words=num_words, oov_token=oov_token, filters='', lower=False)
# se obtienen los tokens para el conjunto de datos, es decir, a cada comando de latex, letra, numero o simbolo
# utilizado se le asigna un numero entero en base a su frecuencia de aparicion
tokenizer.fit_on_texts(data)
# se define la cantidad de palabras o tokens
word_index = tokenizer.word_index
# se guardan los tokens en tokens.json
with open(r"G:\Documents\CROHME\CROHME\CROHME2013_data\TrainINKML\tokens.json", 'w') as f:
    json.dump(word_index, f, indent=4)
# se convierten los labels a tokens para cada archivo inkml
# por ejemplo ['a','b','c'] se convierte a [1,2,3] asumiendo que estos son sus tokens
train_sequences = tokenizer.texts_to_sequences(data)
# se obtiene la longitud del label mas grande
maxlen = max([len(x) for x in train_sequences])
# se asocian los tokens a cada archivo haciendo un pad hacia la maxima longitud
# tal que todos los archivos tenga por label una lista de la misma longitud, rellenando con ceros
# aquellos tokens que se encuentran en una longitud mayor al verdadero para determinado archivo
train_padded = pad_sequences(train_sequences, padding=pad_type, truncating=trunc_type, maxlen=maxlen)

<start> S = ( \sum _ { i = 1 } ^ { n } \theta _ i - ( n - 2 ) \pi ) r ^ 2 <end>


In [72]:
tokens = {}
files_imgs = {}
with open(r"G:\Documents\CROHME\CROHME\CROHME2013_data\TrainINKML\tokens.json", 'r') as f:
    tokens = json.load(f)
tokensPerFile = {}
items = {}
with open(r"G:\Documents\CROHME\CROHME\CROHME2013_data\TrainINKML\labels.json", 'r') as f:
    items = json.load(f)
# se guardab los tokens por archivo inkml haciendo referencia ahora a la imagen 
# generada a partir del archvio inkml
for key in items:
    newKey = LABELS_PATH +'\\images'+ key[len(LABELS_PATH)+11:][:-6]+'.jpg'
    tokensPerFile[newKey] = []
    files_imgs[newKey] = key
    for command in items[key]:
        tokensPerFile[newKey].append(tokens[command])
with open(r"G:\Documents\CROHME\CROHME\CROHME2013_data\TrainINKML\labelsPerFile.json", 'w') as f:
    json.dump(tokensPerFile, f, indent=4)
with open(r"G:\Documents\CROHME\CROHME\CROHME2013_data\TrainINKML\files_img.json", 'w') as f:
    json.dump(files_imgs, f, indent=4)

In [3]:
# se define la arquitectura del encoder en base a Zhang (2017)
class FCN_encoder(tf.keras.Model):
    def __init__(self, dropout_rate = 0.2):
        super(FCN_encoder, self).__init__()
        # super dentro del constructor permite que la clase herede y se convierta en un objeto de Keras
        
        # bloque de convolucion 1, 32 filtros
        self.conv_1_1 = tf.keras.layers.Conv2D(filters=32, kernel_size=3, strides=1)
        self.batch_1_1 = tf.keras.layers.BatchNormalization()
        self.act_1_1 = tf.keras.layers.Activation('relu')
        self.conv_1_2 = tf.keras.layers.Conv2D(filters=32, kernel_size=3, strides=1)
        self.batch_1_2 = tf.keras.layers.BatchNormalization()
        self.act_1_2 = tf.keras.layers.Activation('relu')
        self.conv_1_3 = tf.keras.layers.Conv2D(filters=32, kernel_size=3, strides=1)
        self.batch_1_3 = tf.keras.layers.BatchNormalization()
        self.act_1_3 = tf.keras.layers.Activation('relu')
        self.conv_1_4 = tf.keras.layers.Conv2D(filters=32, kernel_size=3, strides=1)
        self.drop_1 = tf.keras.layers.Dropout(dropout_rate)
        self.batch_1_4 = tf.keras.layers.BatchNormalization()
        self.act_1_4 = tf.keras.layers.Activation('relu')
        
        # maxpooling para reducir el tamaño
        self.maxPool_1 = tf.keras.layers.MaxPooling2D()
        
        # bloque convolucional 2, 64 filtros
        self.conv_2_1 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=1)
        self.batch_2_1 = tf.keras.layers.BatchNormalization()
        self.act_2_1 = tf.keras.layers.Activation('relu')
        self.conv_2_2 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=1)
        self.batch_2_2 = tf.keras.layers.BatchNormalization()
        self.act_2_2 = tf.keras.layers.Activation('relu')
        self.conv_2_3 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=1)
        self.batch_2_3 = tf.keras.layers.BatchNormalization()
        self.act_2_3 = tf.keras.layers.Activation('relu')
        self.conv_2_4 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=1)
        self.drop_2 = tf.keras.layers.Dropout(dropout_rate)
        self.batch_2_4 = tf.keras.layers.BatchNormalization()
        self.act_2_4 = tf.keras.layers.Activation('relu')

        self.maxPool_2 = tf.keras.layers.MaxPooling2D()

        self.conv_3_1 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=1)
        self.batch_3_1 = tf.keras.layers.BatchNormalization()
        self.act_3_1 = tf.keras.layers.Activation('relu')
        self.conv_3_2 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=1)
        self.batch_3_2 = tf.keras.layers.BatchNormalization()
        self.act_3_2 = tf.keras.layers.Activation('relu')
        self.conv_3_3 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=1)
        self.batch_3_3 = tf.keras.layers.BatchNormalization()
        self.act_3_3 = tf.keras.layers.Activation('relu')
        self.conv_3_4 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=1)
        self.drop_3 = tf.keras.layers.Dropout(dropout_rate)
        self.batch_3_4 = tf.keras.layers.BatchNormalization()
        self.act_3_4 = tf.keras.layers.Activation('relu')

        self.maxPool_3 = tf.keras.layers.MaxPooling2D()

        self.conv_4_1 = tf.keras.layers.Conv2D(filters=128, kernel_size=3, strides=2)
        self.batch_4_1 = tf.keras.layers.BatchNormalization()
        self.act_4_1 = tf.keras.layers.Activation('relu')
        self.conv_4_2 = tf.keras.layers.Conv2D(filters=128, kernel_size=3, strides=2)
        self.batch_4_2 = tf.keras.layers.BatchNormalization()
        self.act_4_2 = tf.keras.layers.Activation('relu')
        self.conv_4_3 = tf.keras.layers.Conv2D(filters=128, kernel_size=3, strides=2)
        self.batch_4_3 = tf.keras.layers.BatchNormalization()
        self.act_4_3 = tf.keras.layers.Activation('relu')
        self.conv_4_4 = tf.keras.layers.Conv2D(filters=128, kernel_size=3, strides=2)
        self.drop_4 = tf.keras.layers.Dropout(dropout_rate)
        self.batch_4_4 = tf.keras.layers.BatchNormalization()
        self.act_4_4 = tf.keras.layers.Activation('relu')

        self.maxPool_4 = tf.keras.layers.MaxPooling2D()
        
    def call(self, inputs):
        
        x = self.conv_1_1(inputs)
        x = self.batch_1_1(x)
        x = self.act_1_1(x)
        x = self.conv_1_2(x)
        x = self.batch_1_2(x)
        x = self.act_1_2(x)
        x = self.conv_1_3(x)
        x = self.batch_1_3(x)
        x = self.act_1_3(x)
        x = self.conv_1_4(x)
        x = self.drop_1(x)
        x = self.batch_1_4(x)
        x = self.act_1_4(x)
        x = self.maxPool_1(x)
        
        x = self.conv_2_1(x)
        x = self.batch_2_1(x)
        x = self.act_2_1(x)
        x = self.conv_2_2(x)
        x = self.batch_2_2(x)
        x = self.act_2_2(x)
        x = self.conv_2_3(x)
        x = self.batch_2_3(x)
        x = self.act_2_3(x)
        x = self.conv_2_4(x)
        x = self.drop_2(x)
        x = self.batch_2_4(x)
        x = self.act_2_4(x)
        x = self.maxPool_2(x)
        
        x = self.conv_3_1(x)
        x = self.batch_3_1(x)
        x = self.act_3_1(x)
        x = self.conv_3_2(x)
        x = self.batch_3_2(x)
        x = self.act_3_2(x)
        x = self.conv_3_3(x)
        x = self.batch_3_3(x)
        x = self.act_3_3(x)
        x = self.conv_3_4(x)
        x = self.drop_3(x)
        x = self.batch_3_4(x)
        x = self.act_3_4(x)
        x = self.maxPool_3(x)
        
        x = self.conv_4_1(x)
        x = self.batch_4_1(x)
        x = self.act_4_1(x)
        x = self.conv_4_2(x)
        x = self.batch_4_2(x)
        x = self.act_4_2(x)
        x = self.conv_4_3(x)
        x = self.drop_4(x)
        x = self.batch_4_3(x)
        x = self.act_4_3(x)
        x = self.conv_4_4(x)
        x = self.drop_4(x)
        x = self.batch_4_4(x)
        x = self.act_4_4(x)
        x = self.maxPool_4(x)
    
        return x
    
    def model(self):
        input = tf.keras.layers.Input(shape=(512, 1024, 1))
        return tf.keras.Model(inputs = input, outputs = self.call(input) )
print(FCN_encoder().model().summary())

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 512, 1024, 1 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 510, 1022, 32 320         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 510, 1022, 32 128         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 510, 1022, 32 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [4]:
# Variables necesarias, ignorar por ahora
# M = embdedding dimmensionality
# N = GRU dimensionality
# L = number of annotation vectors
# D = dimensionality of annotation vectors
# K = number of words in vocabulary
# N_prime = attention dimentionality

L = 3
D = 128
K = 127
N_prime = 128
N = 64
M = 128

In [5]:
# el modelo de atencion
class Attender(tf.keras.Model):
    def __init__(self):
        super(Attender, self).__init__()
        # se instancia los distintos dense layers parametrizados en Zhang
        self.W_1 = tf.keras.layers.Dense(128)
        self.U_a = tf.keras.layers.Dense(128)
        # la dimension de atencion es 128
        self.V_a = tf.keras.layers.Dense(128)
        #self.F = tf.keras.layers.Conv2D(filters=32, kernel_size=3, strides=2)
    
    def call(self, a, h):
        # se espanden dimensiones para el hidden state
        h_t = tf.expand_dims(h, 1)
        # se calcula el estado intermedio llamando a los dense layers y la funcion de tangente 
        # hiperbolica con la suma de los dos dense layers que reciben como parametro el hidden state
        # y el vector de anotacion
        e_ti = self.V_a( (tf.nn.tanh( self.W_1(h_t) + self.U_a(a))))
        # se aplica la activacion softmax al resultado
        a_ti = tf.nn.softmax(e_ti)
        # se calcula el vector de contexto multiplicando los coeficientes a_ti por el vector de anotacion
        context = a_ti * a
        # se obtiene la suma del resutado de la multiplicacion anterior
        context = tf.reduce_sum(context, axis =1)
        # se regresa el vector de contexto y los coeficientes a_ti
        return context, a_ti       

In [6]:
global_ = None
# se define el modelo decoder que utiliza el gru
class GRU_decoder(tf.keras.Model):
    def __init__(self, dimension, units, label_len):
        super(GRU_decoder, self).__init__()
        # las unidades del gru
        self.units = units
        # la capa de embedding, establecida en Zhang como E
        self.embedding = tf.keras.layers.Embedding(label_len, dimension)
        # la capa del gru, se especifica que se desea el regreso de las secuencias y el ultimo estado calculado
        self.gru = tf.keras.layers.GRU( self.units, return_sequences=True, return_state = True, recurrent_initializer='glorot_uniform')
        # dense layers para el calculo de las probabilidades para cada palabra en el diccionario de palabras
        # es decir, los tokens
        self.fc1 = tf.keras.layers.Dense(128)
        self.fc2 = tf.keras.layers.Dense(128)#,activation='softmax')
        self.fc3 = tf.keras.layers.Dense(128)
        # la ultima capa se define con una activacion softmax
        self.fc4 = tf.keras.layers.Dense(127, activation='softmax')
        # se instancia el modelo de atencion
        self.attention = Attender()
        self.g = None
        
    def call(self, x, a, h):
        # la llamada recibe la entrada del decoder, la salida del encoder y el estado anterior del decoder
        # se llama al modelo de atencion y se recibe el vector de contexto y los coeficientes de atencion
        context_v, attention_pro = self.attention(a,h)
        x = self.embedding(x)
        # se anade el contexto a la entrada del decoder para utilizar el conocimeinto previo
        x = tf.concat([context_v,x], axis =-1)
        # se alimenta al gru con dicha informacion y se obtiene una salida y su ultimo estado
        out, state = self.gru(x)
        self.g = out
        # es necesario calcular las probabilidades condicionales, por lo que se utiliza 
        # lo descrito por Zhang para cada parte de la salida del gru
        a = self.fc1(out[:,0])
        b = self.fc2(out[:,1])
        c = self.fc3(out[:,2])
        # se suman los resultados
        res = a+b+c
        # se obtienen las probabilidades gracias al softmax del ultimo layer
        res = self.fc4(res)
        #print(res.shape)
        #x = tf.argmax(res, 1)
        # se regresan las probabilidades, el estado y los coeficientes de atencion
        return res, state, attention_pro
    
    def reset(self, batch):
        return tf.zeros((batch,3, self.units))

In [7]:
# isntanciacion del encoder
encoder = FCN_encoder()
# isntanciacion del decoder, con valores para la dimension del embedding, del gru y la cantidad de tokens
decoder = GRU_decoder(128,128,len(list(word_index.keys())))
# se define una funcion de optimizacion para el modelo, el cual permite realizar la optimizacion en base
# al gradiente de los errores
optimizer = tf.keras.optimizers.Adam()
# definimos el tipo de calculo para los errores entre los correspondientes labels del ground truth de cada
# imagen y las predicciones
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

# se define una fucnion de loss (perdida o error) que recibe como parametro un label del ground truth por imagen 
# en el batch las matrices de prediccion asociadas a cada una de esas imagenes
def loss_function(real, pred):
    # se realiza una conversion del tipo de dato para las probabilidades de las predicciones
    pred = tf.cast(pred, tf.float32)
    print(real, pred)
    # se establece una mascara para aquellas imagenes cuyo label sea 0, es decir, que
    # es el padding agregado y que de esta forma no se considere su error para el error general
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    # se calcula el error
    loss_ = loss_object(real, pred)
    mask = tf.cast(mask, dtype=loss_.dtype)
    # se anade la mascara al los obtenido
    loss_ *= mask
    # se retorna la media de las perdidas para las imagenes en el batch
    return tf.reduce_mean(loss_)

In [8]:
data = {}
# se obtienen los tokens por imagen
LABELS_PATH = r"G:\Documents\CROHME\CROHME\CROHME2013_data\TrainINKML"
with open(LABELS_PATH+'\\labelsPerFile.json',  'r') as f:
    data = json.load(f)
    f.close()

In [9]:
files_img = {}
# se obtienen los nombres de las imagenes
with open(r"G:\Documents\CROHME\CROHME\CROHME2013_data\TrainINKML\files_img.json", 'r') as f:
    files_img = json.load(f)
    f.close()
keys = list(files_img.keys())
# se obtiene el dataset a partir del diccionario que contiene los tokens por archivo
list_ds = tf.data.Dataset.list_files(list(data.keys()))

# funcion que obtiene la imagen y su label, tiene como parametro el path de la imagen
def getProcessedImages(f):
    index = keys.index(f)
    # se lee la imagen
    image = tf.io.read_file(f)
    # se decodifica el formato jpg y se establece como imagen en un solo canal, es decir
    # blanco y negro
    image = tf.image.decode_jpeg(image, channels = 1)  
    # se convierte su tipo de dato a flotante
    image = tf.image.convert_image_dtype(image, tf.float32)
    # se redimensiona para que concuerde con el modelo del encoder
    image = tf.image.resize(image, [512, 1024] ) 
    # se retorna la imagen (su matriz de pixels) y su label
    return image, train_padded[index] #[data[str(tf.constant(f).numpy())[2:-1].replace('\\\\', '\\') ]]
    
# define el dataset a utilizar como un mapeo de los archvios dentro de list_ds que manda llamar a la funcion getProcessedImages
labeledDataset = list_ds.map(lambda x: tf.py_function(getProcessedImages, [x], [tf.float32, tf.int32]) ,num_parallel_calls=tf.data.AUTOTUNE)

# solo para verificar que funciona, imprime las dimensioens de la imagen y su label
for image, label in labeledDataset.take(1):
    print("Image shape: ", image.numpy().shape)
    print("Label: ", label.numpy())

Image shape:  (512, 1024, 1)
Label:  [ 4  2 48  3 11  2 20  2 26  3  3  5  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]


In [10]:
# se establece el tamano de batch como 8, es decir, se ingresara al entrenamiento paquetes de 8 imagenes
labeledDataset = labeledDataset.batch(8)
#labeledDataset = labeledDataset.prefetch(buffer_size = tf.data.AUTOTUNE)

'for image, label in labeledDataset.batch(64).take( 1 ):\n    print("Image shape: ", image.numpy().shape)\n    print("Label: ", label.numpy())'

In [11]:
import random
# se define la funcion que entrena al modelo de encoder-decoder
# recibe como parametros el batch de imagenes y su ground truth (labels)
def train(image, groundTruth):
    loss = 0
    # el estado del gru se inicializa como ceros
    hidden = decoder.reset(groundTruth.shape[0])
    # el input del decoder se inicializa como un tensor con valores para el primer token en todas las imagenes
    input_decoder = tf.constant([[word_index['<start>']]*3] * groundTruth.shape[0])
    #input_decoder = tf.expand_dims([[word_index['<start>']]*3] * groundTruth.shape[0], 1)
    #print(input_decoder.shape, hidden.shape)
    # ciclo que permite el entrenamiento al generar un entorno donde las variables de entrenamiento son 'vigiladas'
    # durante el entrenamiento para poder corregirlas y ajusstar el modelo
    with tf.GradientTape() as tape:
        # se llama al encoder con la imagen la cual regresa el feature map con las caracteristicas extraidas de la imagen
        feature_map = encoder(image)
        # se itera por la cantidad de posibles palabras en un groundTruth
        for i in range(1, groundTruth.shape[1]):
            print(i)
            # para cada palabra se verifica si su tensor corresponde a solo valores en cero, para lo cual se rompe el ciclo
            # terminando el entrenamiento de este batch
            sum_ = tf.reduce_sum(groundTruth[:,i])
            allZero = tf.equal(sum_, 0)
            if (allZero):
                break
            # si existe por lo menos un label por predecir entonces se llama al decoder envindole 
            # su respectivo input, las salidas del encoder y el hidden state anterior
            pred, hidden, alphas = decoder(input_decoder, feature_map, hidden)
            #print(groundTruth[:,i], pred.shape, hidden.shape, alphas.shape)
            # se calcula el error de las predicciones y se suma al actual para el batch
            loss += loss_function(groundTruth[:, i], pred)
            print("loss:", loss)
            # la siguiente entrada para el decoder son las anotaciones para la iteracion actual
            # es decir, la anterior para la siguiente interacion (h-1)
            dec_input = tf.expand_dims(groundTruth[:, i], 1)
    # una vez que termina la entrada del batch se calcula el error total con el error obtenido
    total_loss = (loss / int(groundTruth.shape[1]))
    # se obtienen las variables a las que se les puede modificar los parametros para ajustarlas (entrenarlas)
    trainable_variables = encoder.trainable_variables + decoder.trainable_variables
    # se obtiene el gradiente de error en base al error obtenido
    gradients = tape.gradient(loss, trainable_variables)
    # se realiza el ajuste en base al gradiente de error
    optimizer.apply_gradients(zip(gradients, trainable_variables))
    # se regresa el error y el error total
    return loss, total_loss

In [17]:
# funcion que itera sobre el dataset para entrenarlo
def trainData(dataset, EPOCHS = 1):
    print(len(dataset))
    # se itera por las epocas de entrenamiento
    for epoch in range(EPOCHS):
        total_loss = 0
        num = 0
        # para cada batch de images en el dataset
        for (batch, (img_tensor, target)) in enumerate(dataset):
            # se llama a la funcion de entrenamiento para cada batch enviando el tensor de imagenes y de labels
            # obteniendo los erroes
            batch_loss, t_loss = train(img_tensor, target)
            total_loss += t_loss
            num +=1
            print(batch)
            if (batch+1) % 10 == 0:
                break
                #average_batch_loss = batch_loss.numpy()/int(target.shape[1])
        print('Epoch {0:d}/{1:d}'.format(epoch+1, EPOCHS))
        print('===============>  train-loss=%.3f' % (total_loss/num))

In [18]:
# se llama la funcion principal de entrenamiento
trainData(labeledDataset)

1023
(8, 3) (8, 3, 128)
1
tf.Tensor([23 21 19 16 18 16  2 13], shape=(8,), dtype=int32) tf.Tensor(
[[0.00775045 0.00789502 0.00859775 ... 0.00788662 0.00746669 0.00810054]
 [0.00775045 0.00789502 0.00859775 ... 0.00788662 0.00746669 0.00810054]
 [0.00775045 0.00789501 0.00859775 ... 0.00788662 0.00746669 0.00810054]
 ...
 [0.00775045 0.00789501 0.00859775 ... 0.00788662 0.00746669 0.00810055]
 [0.00775045 0.00789502 0.00859775 ... 0.00788662 0.00746669 0.00810055]
 [0.00775045 0.00789501 0.00859775 ... 0.00788662 0.00746669 0.00810054]], shape=(8, 127), dtype=float32)
loss: tf.Tensor(4.811062, shape=(), dtype=float32)
2
tf.Tensor([ 2 92 32  2 55  2 48 13], shape=(8,), dtype=int32) tf.Tensor(
[[0.00775215 0.00789048 0.00860424 ... 0.0078832  0.00746501 0.008095  ]
 [0.00775218 0.00789048 0.00860425 ... 0.0078832  0.00746498 0.00809498]
 [0.00775215 0.00789048 0.00860422 ... 0.0078832  0.007465   0.00809501]
 ...
 [0.00775212 0.00789046 0.0086042  ... 0.00788321 0.00746498 0.00809503]
 [

In [ ]:
# ignorar todo a partir de aqui, es para hacer pruebas jeje :)

'''
def preTraining(dataSet, cache=True, shuffle_buffer_size=1000):
    dataSet = dataSet.shuffle( buffer_size=shuffle_buffer_size )
    #dataSet = dataSet.repeat()
    dataSet = dataSet.batch(16) #batch size =32
    #dataSet = dataSet.prefetch(buffer_size = tf.data.AUTOTUNE)
    return dataSet
trainDS = preTraining(labeledDataset)
#image_batch, label_batch = next(iter(trainDS))
#show_batch(image_batch.numpy(), label_batch.numpy())'''

In [14]:
# convert to grayscale
PATH = r"G:\\Documents\\CROHME\\CROHME\\CROHME2013_data\\TrainINKML\\images\\expressmatch\\101_alfonso.jpg"
image = cv.imread(PATH)
resized = cv.resize(image, (1024,512), interpolation = cv.INTER_AREA)
 
print('Resized Dimensions : ',resized.shape)
 
cv.imshow("Resized image", resized)
cv.waitKey(0)
cv.destroyAllWindows()
#grayImage = cv.cvtColor(image, cv.COLOR_BGR2GRAY)


Resized Dimensions :  (512, 1024, 3)


In [ ]:
path = r"G:\Documents\CROHME\CROHME\CROHME2013_data\TrainINKML\images\expressmatch\65_alfonso.jpg"

In [6]:
a = tf.zeros((16,1,3, 128))
print(a.shape)

(16, 1, 3, 128)


In [2]:
from keras.models import Sequential 
from keras.layers import Activation, Dense 
model = Sequential() 
layer_1 = Dense(128, input_shape = (56,)) 

model.add(layer_1) 
print(layer_1.input_shape) 
print(layer_1.output_shape )

(None, 56)
(None, 128)


In [17]:
t = decoder.g

class test(tf.keras.Model):
    def __init__(self):
        super(test, self).__init__()
        self.fc1 = tf.keras.layers.Dense(128)
        self.fc2 = tf.keras.layers.Dense(128)#,activation='softmax')
        self.fc3 = tf.keras.layers.Dense(128)
        self.fc4 = tf.keras.layers.Dense(127, activation='softmax')
    
    def call(self, a, b, c):
        a = self.fc1(a)
        b = self.fc2(b)
        c = self.fc3(c)
        res = a+b+c
        res = self.fc4(res)
        return res
tst = test()
res = tst(t[:,0], t[:,1], t[:,2])
print(res[1], max(res[1]))

tf.Tensor(
[0.0077122  0.00798441 0.0082583  0.00813114 0.00758686 0.00765422
 0.00819508 0.00801731 0.00778867 0.00793868 0.00817909 0.0077188
 0.00780235 0.00792794 0.00782531 0.00801626 0.00759206 0.00791037
 0.00784708 0.00803427 0.00783526 0.00810021 0.00777087 0.00752414
 0.00764606 0.00757418 0.007841   0.00777249 0.00772197 0.00759741
 0.0077856  0.0078179  0.00805625 0.00800839 0.00779082 0.00805504
 0.00801177 0.00805107 0.00823483 0.0081527  0.00743428 0.00756936
 0.00782649 0.00787631 0.00812574 0.00805205 0.00788147 0.00777273
 0.00788906 0.00815082 0.00745555 0.00793089 0.00765897 0.00804335
 0.00789843 0.0077011  0.00802226 0.00792664 0.0081496  0.00784043
 0.00800649 0.0077229  0.00779958 0.00769156 0.00797984 0.0078022
 0.00766275 0.00797059 0.00790793 0.0080414  0.00796528 0.0081026
 0.00842888 0.00783339 0.00775519 0.00776263 0.0078705  0.0076692
 0.00840811 0.00759783 0.00786747 0.00764156 0.00765676 0.00792471
 0.00792545 0.00794002 0.00792474 0.00750257 0.00773712

In [30]:
'''import tensorflow_probability as tfp
resp = tfp.distributions.Multinomial(probs=[.2,.5,.6], num_samples=1)
print(resp)'''
'''import tensorflow as tf
samples = tf.constant([[.25,.26,.6], [.85, .9, 0.25], [.3, .8, .4]])
shape = samples.shape
for i in range(shape[1]):
print(samples)
print(tf.reduce_max(tf.compat.v1.multinomial(samples, 1), axis=1))'''
res_words = tf.reduce_max(res, axis=1)


print(tf.reduce_max(res, axis=1))
#print(tf.compat.v1.multinomial(logits=samples, num_samples=1))

tf.Tensor(
[0.00836815 0.00836697 0.00836866 0.00836738 0.00836732 0.0083674
 0.00836789 0.0083669 ], shape=(8,), dtype=float32)


In [18]:
B = tf.constant([[2, 20, 30, 3, 6], [3, 11, 16, 1, 8],
                 [14, 45, 23, 5, 27]])
print(tf.argmax(res, 1), tf.transpose(tf.argmax(res, 1)))

tf.Tensor([72 72 72 72 72 72 72 72], shape=(8,), dtype=int64) tf.Tensor([72 72 72 72 72 72 72 72], shape=(8,), dtype=int64)
